In [ ]:
#widen display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time

In [2]:
nba_stats_base_url = 'https://www.nba.com/stats/teams/traditional?PerMode=Totals'

In [3]:
seasons = ['&Season=2023-24', '&Season=2022-23', '&Season=2021-22', '&Season=2020-21', '&Season=2019-20'
          , '&Season=2018-19', '&Season=2017-18', '&Season=2016-17', '&Season=2015-16', '&Season=2014-15', '&Season=2013-14'
          , '&Season=2012-13', '&Season=2011-12', '&Season=2010-11', '&Season=2009-10', '&Season=2008-09', '&Season=2007-08'
          , '&Season=2006-07', '&Season=2005-06', '&Season=2004-05', '&Season=2003-04', '&Season=2002-03'
          , '&Season=2001-02', '&Season=2000-01']

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#Vist first season to create dataframe - don't feel like manually creating the dataframe columns :)
browser.visit(nba_stats_base_url)
time.sleep(3)

html = browser.html
stats_html = BeautifulSoup(html, 'html.parser')

tables = pd.read_html(str(stats_html))

all_seasons_stats_df = tables[2]

all_seasons_stats_df.insert(54, "SEASON", "2023-24", True) 

In [6]:
all_seasons_stats_df.head()

,Unnamed: 0,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,...,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK,SEASON
0,1,Boston Celtics,79,62,17,0.785,3822,9515,3456,7117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
1,2,Denver Nuggets,79,55,24,0.696,3797,9056,3469,7010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
2,2,Minnesota Timberwolves,79,55,24,0.696,3817,8942,3268,6743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
3,4,Oklahoma City Thunder,79,54,25,0.684,3817,9460,3505,7037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
4,5,LA Clippers,79,51,28,0.646,3797,9159,3345,6813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24


In [7]:
#loop through rest of seasons and append dataframes
for season in seasons:
    nba_season_stats_url = nba_stats_base_url + season

    browser.visit(nba_season_stats_url)
    time.sleep(3)
    
    html = browser.html
    stats_html = BeautifulSoup(html, 'html.parser')
    
    tables = pd.read_html(str(stats_html))
    
    this_season_stats_df = tables[2]
    
    #add the season for this set of stats
    this_season_stats_df.insert(54, "SEASON", season.strip('&Season='), True) 
    
    all_seasons_stats_df = all_seasons_stats_df.append(this_season_stats_df, ignore_index=True)

In [8]:
#drop unwanted columns
all_seasons_stats_df.drop(all_seasons_stats_df.iloc[:, 28:54], inplace = True, axis = 1)
all_seasons_stats_df.drop(all_seasons_stats_df.columns[0], inplace = True, axis=1)

In [9]:
all_seasons_stats_df.head()

,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,SEASON
0,Boston Celtics,79,62,17,0.785,3822,9515,3456,7117,48.6,...,3662,2119,944,528,510,289,1277,1373,896,2023-24
1,Denver Nuggets,79,55,24,0.696,3797,9056,3469,7010,49.5,...,3512,2321,995,560,439,383,1432,1415,408,2023-24
2,Minnesota Timberwolves,79,55,24,0.696,3817,8942,3268,6743,48.5,...,3462,2110,1114,623,476,359,1481,1560,554,2023-24
3,Oklahoma City Thunder,79,54,25,0.684,3817,9460,3505,7037,49.8,...,3289,2123,1002,663,521,410,1500,1504,503,2023-24
4,LA Clippers,79,51,28,0.646,3797,9159,3345,6813,49.1,...,3384,2028,1040,609,395,366,1475,1497,297,2023-24


In [10]:
all_seasons_stats_df = all_seasons_stats_df.rename(columns={'TEAM':'team','GP':'games_played','W':'wins','L':'losses','WIN%':'win_percentage','MIN':'minutes','PTS':'points','FGM':'field_goals_made','FGA':'field_goals_attempted','FG%':'field_goal_percentage','3PM':'three_pointers_made','3PA':'three_pointers_attempted','3P%':'three_point_percentage','FTM':'free_throws_made','FTA':'free_throw_attempted','FT%':'free_throw_percentage','OREB':'offensive_rebounds','DREB':'defensive_rebounds','REB':'rebounds','AST':'assists','TOV':'turnovers','STL':'steals','BLK':'blocks','BLKA':'blocks_attempted','PF':'personal_fouls','PFD':'personal_fouls_drawn','+/-':'plus_minus','SEASON':'season'})

In [11]:
all_seasons_stats_df.index.name = 'teamstatspk'

In [12]:
all_seasons_stats_df.to_csv('data/nba_team_stats_00_to_23.csv')

In [ ]:
all_seasons_stats_df.head()